In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
input_tensor  = tf.placeholder(shape=(None, 28, 28, 1), dtype="float32", name="input_tensor")
output_tensor = tf.placeholder(shape=(None, 10), dtype="float32", name="input_tensor")

In [4]:
layer_1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (1, 1), activation = "relu")(input_tensor)
layer_2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2, 2), activation = "relu")(layer_1)
layer_3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2, 2), activation = "relu")(layer_2)
layer_4 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (2, 2), activation = "relu")(layer_3)
layer_5 = tf.keras.layers.AveragePooling2D(pool_size= 2, strides =1)(layer_4) 
layer_6 = tf.keras.layers.Dense(units= 10)(layer_5)
layer_7 = tf.reshape(layer_6,shape = (-1, 10))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# layer_1 = tf.keras.layers.c=Conv2D(input_tensor, kernel_size=(3,3), filters=32, strides=(1, 1), activation="relu")
# layer_2 = tf.layers.conv2d(layer_1, kernel_size=(3,3), filters=32, strides=(2, 2), activation="relu")
# layer_3 = tf.layers.conv2d(layer_2, kernel_size=(3,3), filters=64, strides=(2, 2), activation="relu")
# layer_4 = tf.layers.conv2d(layer_3, kernel_size=(3,3), filters=128, strides=(2, 2), activation="relu")
# layer_5 = tf.layers.average_pooling2d(layer_4, pool_size=2,strides=1)
# layer_6 = tf.layers.dense(layer_5, units=10)
# layer_7 = tf.reshape(layer_6, shape=(-1, 10))

In [6]:
print(layer_1)
print(layer_2)
print(layer_3)
print(layer_4)
print(layer_5)
print(layer_6)
print(layer_7)

Tensor("conv2d/Relu:0", shape=(?, 26, 26, 32), dtype=float32)
Tensor("conv2d_1/Relu:0", shape=(?, 12, 12, 32), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(?, 5, 5, 64), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(?, 2, 2, 128), dtype=float32)
Tensor("average_pooling2d/AvgPool:0", shape=(?, 1, 1, 128), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 1, 1, 10), dtype=float32)
Tensor("Reshape:0", shape=(?, 10), dtype=float32)


In [7]:
prediction = layer_7

In [8]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=output_tensor, logits=prediction) # tf.reduce_mean(tf.squared_difference(output_tensor, prediction))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(output_tensor, 1)), "float32"))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
optimizer = tf.train.AdamOptimizer()
loss_optimization = optimizer.minimize(loss)

In [10]:
images_list = []
for root, folder, files in os.walk("/home/grace/Documents/Mnist_DataSet/Train_Data/"):
    for file in files :
        images_list.append(root + file)
print("No of Images : ", len(images_list))

No of Images :  42000


In [11]:
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)

In [12]:
num_epochs = 5
batch = 30
total_images = len(images_list)
for epoch in range(num_epochs):
    acc_list = []
    loss_list = []
    for i in range(0, total_images-batch, batch):
        image_batch = []
        label_batch = []
        for image in images_list[i:i+batch]:
            single_image = cv2.imread(image, 0)/255.0 - 0.5
            single_image = np.reshape(single_image, (28,28,1))
            image_batch.append(single_image)
            label = int(image.split("/")[-1].split("_")[0])
            label_batch.append(np.eye(10)[label])
        label_batch = np.array(label_batch, dtype=np.float32)
        
        if epoch == 0:            
            acc, loss_val = sess.run([accuracy, loss], feed_dict={input_tensor :image_batch, output_tensor : label_batch })
            acc_list.append(acc)
            loss_list.append(loss_val)
        else : 
            opt, acc, loss_val = sess.run([loss_optimization, accuracy, loss], feed_dict={input_tensor :image_batch, output_tensor : label_batch })
            acc_list.append(acc)
            loss_list.append(loss_val)
    print("Epoch : " + str(epoch) + " Accuracy : " + str(np.average(acc)) + " Loss : " + str(np.average(loss_list)))

Epoch : 0 Accuracy : 0.16666667 Loss : 0.6911525
Epoch : 1 Accuracy : 0.96666664 Loss : 0.07912721
Epoch : 2 Accuracy : 0.96666664 Loss : 0.020430183
Epoch : 3 Accuracy : 0.96666664 Loss : 0.014013313
Epoch : 4 Accuracy : 1.0 Loss : 0.0102731595
